### Importing Packages

In [136]:
import json
from openai import AzureOpenAI
import requests
from bs4 import BeautifulSoup

### Defining custom functions

In the cell below, custom functions are created

In [137]:
def custom_alpha(number_list):
    # Custom function
    return (number_list[0] + number_list[1]) * (
        2 * number_list[0] + (number_list[0] * number_list[1])
    )


def custom_beta(input_string):
    # Custom function

    working_string = input_string.lower()

    # Split the string into words based on spaces
    words = working_string.split()
    # Sort the words
    sorted_words = sorted(words)
    # Reconstruct the string with spaces between sorted words
    result = " ".join(sorted_words)

    return result


def fetch_word_of_the_day():
    url = "https://www.merriam-webster.com/word-of-the-day"
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")

        # The word of the day and definition are usually located in these tags
        word_element = soup.find("h2", class_="word-header-txt")

        if word_element:
            word = word_element.text.strip()

            return word
        else:
            return None
    else:
        return None

### Setting up the client

In [138]:
client = AzureOpenAI(
    azure_endpoint="https://giles-openai-test.openai.azure.com/",
    api_key="27e9399d48b64ce99400c08715dcec36",
    api_version="2023-03-15-preview",
)

deployment_name = "giles-gpt-4o"

### System prompt

A system prompt is added to initialize how the model should "act"

In [139]:
system_prompt = """
You are an AI assistant. You are to assist the user on their requests. 

You also have access to certain functions. Please utilize them when the user has specifically requested them to be used.
If the user has NOT specified them, do NOT use functions. Simply respond normally as if you don't have access to the functions.

If the function name is similar to a phrase, please consult the condition of the function first. There may be specific conditions listed in tools when defining such function.
If the conditions are not met, do NOT use the function.

An example would be the phrase word of the day. You have access to fetch_word_of_the_day tool but only use it if the user asked for word of the day with Merriam-Webster present in the prompt.
"""

message_list = [
    {
        "role": "system",
        "content": system_prompt,
    }
]

### Defining of tools

Here are the tools accessible by the model

In [140]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "custom_alpha",
            "description": "Perform the Custom Alpha function on a list of numbers provided",
            "parameters": {
                "type": "object",
                "properties": {
                    "numbers": {
                        "type": "array",
                        "items": {"type": "number"},
                        "description": "A list of numbers",
                    },
                },
                "required": ["numbers"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "custom_beta",
            "description": "Perform the Custom Beta function on a string",
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {
                        "type": "string",
                        "description": "A string",
                    },
                },
                "required": ["text"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "fetch_word_of_the_day",
            "description": "Fetch the word of the day from Merriam-Webster. Use only when user SPECIFICALLY ASKED for it to be from Merriam-Webster. Otherwise, give a random word",
            "parameters": {
                "type": "object",
                "properties": {
                    "placeholder": {
                        "type": "string",
                        "description": "Placeholder",
                    },
                },
                "required": ["placeholder"],
            },
        },
    },
]

### User interaction

In [141]:
user_prompt = str(input("Enter your prompt: "))

print(f"\n{user_prompt}\n")

message_list.append(
    {
        "role": "user",
        "content": user_prompt,
    }
)


Tell me a random fun fact about birds. Perform custom alpha on 5,1. Do custom beta on "birds are great". Give me a random word of the day. Also give me the word of the day from merriam



### Response

The prompt is sent to the model

In [142]:
response = client.chat.completions.create(
    model=deployment_name,
    messages=message_list,
    tools=tools,
    tool_choice="auto",
)

response_message = response.choices[0].message
message_list.append(response_message)

tool_call_presence = False

if response_message.tool_calls:

    tool_call_presence = True

    for tool_call in response_message.tool_calls:

        if tool_call.function.name == "custom_alpha":

            function_args = json.loads(tool_call.function.arguments)
            print(f"Function arguments: {function_args}")

            function_response = custom_alpha(
                number_list=function_args.get("numbers", [])
            )

            print(function_response)
            function_response_string = f"The result of the function is {function_response}. Please convey the result in a conversational manner"

            message_list.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": "custom_alpha",
                    "content": function_response_string,
                }
            )

        if tool_call.function.name == "custom_beta":

            function_args = json.loads(tool_call.function.arguments)
            print(f"Function arguments: {function_args}")

            function_response = custom_beta(input_string=function_args.get("text", []))

            print(function_response)
            function_response_string = f"The result of the function is {function_response}. Please convey the result in a conversational manner"

            message_list.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": "custom_beta",
                    "content": function_response_string,
                }
            )

        if tool_call.function.name == "fetch_word_of_the_day":

            function_response = fetch_word_of_the_day()

            print(function_response)
            function_response_string = f"The result of the function is {function_response}. Please convey the result in a conversational manner"

            message_list.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": "fetch_word_of_the_day",
                    "content": function_response_string,
                }
            )

else:
    tool_call_presence = False

Function arguments: {'numbers': [5, 1]}
90
Function arguments: {'text': 'birds are great'}
are birds great
fret


### Final Response

If no function is called, we get the regular completion, where the model response to the prompt.

If a function is called, the result of the function must be sent back to the model so that the model can return it in a conversational manner

In [143]:
if not tool_call_presence:
    print("No Function Called\n\n\n\n")
    print(f"User prompt: {user_prompt}")
    print(f"Model response: {response_message.content}")

else:
    print("Function Called\n\n\n\n")
    response = client.chat.completions.create(
        model=deployment_name,
        messages=message_list,
    )

    print(f"User prompt: {user_prompt}")
    print(f"Model response: {response.choices[0].message.content}")


Function Called




User prompt: Tell me a random fun fact about birds. Perform custom alpha on 5,1. Do custom beta on "birds are great". Give me a random word of the day. Also give me the word of the day from merriam
Model response: Here's a random fun fact about birds: Did you know that owls are the only birds that can see the color blue?

For the custom operations you asked for:
- The result of `custom_alpha` on 5 and 1 is **90**.
- The result of `custom_beta` on "birds are great" is **"are birds great"**.

As for the word of the day, a random word of the day is **"fret"**.

You requested the word of the day from Merriam-Webster specifically, but I can only provide it if you explicitly mention Merriam-Webster in your prompt. If you still need it, please let me know!


### Extra

In [144]:
for message in message_list:
    print(f"\n{message}\n")


{'role': 'system', 'content': "\nYou are an AI assistant. You are to assist the user on their requests. \n\nYou also have access to certain functions. Please utilize them when the user has specifically requested them to be used.\nIf the user has NOT specified them, do NOT use functions. Simply respond normally as if you don't have access to the functions.\n\nIf the function name is similar to a phrase, please consult the condition of the function first. There may be specific conditions listed in tools when defining such function.\nIf the conditions are not met, do NOT use the function.\n\nAn example would be the phrase word of the day. You have access to fetch_word_of_the_day tool but only use it if the user asked for word of the day with Merriam-Webster present in the prompt.\n"}


{'role': 'user', 'content': 'Tell me a random fun fact about birds. Perform custom alpha on 5,1. Do custom beta on "birds are great". Give me a random word of the day. Also give me the word of the day from